In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, RidgeCV
from sklearn.metrics import (
 r2_score, mean_absolute_error, mean_squared_error,
)

# Configuration esthétique pour les graphiques
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
import mlflow
from scipy import stats
from equipy.fairness import FairWasserstein
from equipy.metrics import unfairness
from models_and_metrics import *
from real_datasets import *
from scipy.stats import kstest, ks_2samp



In [20]:
chemin_df_labeled = "gossis_labeled.csv"
chemin_df_unlabeled = "gossis_unlabeled.csv"
df_labeled = pd.read_csv(chemin_df_labeled)
df_unlabeled = pd.read_csv(chemin_df_unlabeled)

In [21]:
labeled_data, blackbox_scores_labeled, s_labeled, unlabeled_data, blackbox_scores_unlabeled, s_unlabeled = load_Gossis_data.load_gossis_data(
    labeled=df_labeled,
    unlabeled=df_unlabeled,
    target_feature="hospital_death",
    sensitive_group="African American",
    blackbox_feature="apache_4a_hospital_death_prob"
)

STEP 1 – INITIAL LOAD & CLEANING

Initial shape: Labeled=(91713, 180), Unlabeled=(39308, 180)

Filtering columns (≤10 % NA) – keeping 81 features.
→ Shape after NA cleaning: Labeled=(56451, 81), Unlabeled=(23100, 80)

Encoding sensitive group 'African American' in column 'ethnicity' …
→ Unique values: [-1  1]

Extracting pre‑computed scores from 'apache_4a_hospital_death_prob' …


In [22]:
def identify_variable_types(df, columns=None, discrete_threshold=10, unique_ratio_threshold=0.05):
    """
    Identifie si les variables sont continues ou discrètes.
    
    Paramètres:
    - df: DataFrame pandas
    - columns: Liste des colonnes à vérifier (None = toutes les colonnes)
    - discrete_threshold: Nombre maximum de valeurs uniques pour considérer une variable comme discrète
    - unique_ratio_threshold: Ratio minimum de valeurs uniques/total pour considérer une variable comme continue
    
    Retourne:
    - Un dictionnaire avec les résultats et un DataFrame résumant les caractéristiques
    """
    if columns is None:
        columns = df.columns
    
    results = {
        'continuous': [],
        'discrete': [],
        'categorical': [],
        'binary': [],
        'summary': []
    }
    
    for col in columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            results['categorical'].append(col)
            continue
            
        n_unique = df[col].nunique()
        n_total = len(df[col])
        unique_ratio = n_unique / n_total
        
        # Collecter les statistiques
        summary = {
            'column': col,
            'dtype': df[col].dtype,
            'n_unique': n_unique,
            'unique_ratio': unique_ratio,
            'min': df[col].min(),
            'max': df[col].max(),
            'has_decimals': any(x % 1 != 0 for x in df[col].dropna().sample(min(1000, len(df[col]))).values)
        }
        results['summary'].append(summary)
        
        # Classifier la variable
        if n_unique == 2:
            results['binary'].append(col)
            results['discrete'].append(col)
        elif n_unique <= discrete_threshold or unique_ratio < unique_ratio_threshold:
            results['discrete'].append(col)
        else:
            # Vérifier si la variable a des valeurs décimales
            if summary['has_decimals']:
                results['continuous'].append(col)
            else:
                # Si beaucoup de valeurs uniques mais toutes entières, c'est probablement un ID ou une variable discrète
                if n_unique > 100:
                    results['continuous'].append(col)  # Probablement un ID numérique
                else:
                    results['discrete'].append(col)
    
    # Créer un DataFrame de résumé
    summary_df = pd.DataFrame(results['summary'])
    if not summary_df.empty:
        summary_df['classification'] = summary_df['column'].apply(
            lambda x: 'binary' if x in results['binary'] 
                     else 'discrete' if x in results['discrete'] 
                     else 'continuous' if x in results['continuous']
                     else 'categorical'
        )
    
    return results, summary_df

# Exemple d'utilisation
# results, summary_df = identify_variable_types(df, columns=['age', 'income', 'gender'])
# print(summary_df)


In [23]:
def select_uncorrelated_features(df, threshold=0.8):
    """
    Sélectionne les variables qui ne sont pas fortement corrélées entre elles.
    
    Args:
        df: DataFrame contenant les variables
        threshold: Seuil de corrélation (défaut: 0.8)
        
    Returns:
        Liste des noms de variables à conserver
    """
    # Calculer la matrice de corrélation
    corr_matrix = df.corr().abs()
    
    # Créer une matrice triangulaire supérieure
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Identifier les colonnes à supprimer (corrélées au-dessus du seuil)
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Colonnes à conserver
    to_keep = [column for column in df.columns if column not in to_drop]
    
    return to_keep

In [24]:
list_continuous_candidates = labeled_data.describe().columns.to_list()
results, summary_df = identify_variable_types(labeled_data, columns=list_continuous_candidates)
list_columns_to_test = summary_df[summary_df['n_unique']>70]['column'].to_list()
numerical_features_to_test = [i for i in list_columns_to_test if i not in ["apache_2_bodysystem", "icu_admit_source",'encounter_id','patient_id', 'hospital_id']]
list_less_correlated_var = select_uncorrelated_features(labeled_data[numerical_features_to_test])

In [25]:
# Listes de variables sélectionnées
categorical_features = ["apache_2_bodysystem", "icu_admit_source"]
ordinal_features = ["gender"]
numerical_features = list_less_correlated_var
# [
#     "age", "bmi", "immunosuppression",
#     "d1_diasbp_noninvasive_max", "d1_sysbp_noninvasive_max",
#     "d1_heartrate_min", "resprate_apache", "d1_sysbp_min",
#     "d1_spo2_min", "d1_glucose_max", "pre_icu_los_days", "map_apache","apache_4a_hospital_death_prob"
# ]
sensitive_feature="ethnicity"
target_feature = "hospital_death"

all_selected_features = categorical_features + ordinal_features + numerical_features + [sensitive_feature, target_feature]
df_analysis = labeled_data[all_selected_features].copy()

In [26]:
labeled_data_preprocessed, unlabeled_data_preprocessed, data_orig_names = load_Gossis_data.preprocess_features(
    labeled_data,
    unlabeled_data,
    target_feature,
    sensitive_feature,
    categorical_features,
    ordinal_features,
    numerical_features, # Renommé de no_change_features
    do_scale=True
)


STEP 2 – FEATURE ENGINEERING

Original feature sample:
  apache_2_bodysystem      icu_admit_source gender   age    bmi  height  \
0      Cardiovascular                 Floor      M  68.0  22.73   180.3   
1         Respiratory                 Floor      F  77.0  27.42   160.0   
5          Neurologic  Accident & Emergency      M  67.0  27.56   190.5   

   pre_icu_los_days  apache_3j_diagnosis  heart_rate_apache  map_apache  ...  \
0          0.541667               502.01              118.0        40.0  ...   
1          0.927778               203.01              120.0        46.0  ...   
5          0.000694               403.01              113.0       130.0  ...   

   h1_spo2_min  h1_sysbp_max  d1_creatinine_max  d1_glucose_max  \
0         74.0         131.0               2.51           168.0   
1         70.0          95.0               0.71           145.0   
5         97.0         143.0               0.71           156.0   

   d1_glucose_min  d1_potassium_max  d1_potassium_min

In [27]:
labeled_data_preprocessed.replace({'ethnicity': {1: 2}},inplace=True)
labeled_data_preprocessed.replace({'ethnicity': {-1: 1}},inplace=True)
unlabeled_data_preprocessed.replace({'ethnicity': {1: 2}},inplace=True)
unlabeled_data_preprocessed.replace({'ethnicity': {-1: 1}},inplace=True)
labeled_data_preprocessed['ethnicity'].value_counts()

ethnicity
1    50345
2     6106
Name: count, dtype: int64

In [28]:
list_predictions = ['y_input_reg', 'y_pred_fair', 'y_score_equipy', 'y_pred_riken']
list_description = ['Regression Lineaire Input','Modèle Lineaire Fair','Equipy','Riken']
S_variable='ethnicity'
train_dataset, test_dataset = train_test_split(labeled_data_preprocessed, test_size=0.2)
train_dataset,pool_dataset = train_test_split(train_dataset, test_size=0.2)

In [29]:
bool_columns = unlabeled_data_preprocessed.select_dtypes(include='bool').columns.tolist()
unlabeled_data_preprocessed[bool_columns] = unlabeled_data_preprocessed[bool_columns].astype(int)
bool_columns = labeled_data_preprocessed.select_dtypes(include='bool').columns.tolist()
labeled_data_preprocessed[bool_columns] = labeled_data_preprocessed[bool_columns].astype(int)

In [30]:
y= 'h1_diasbp_max'
S_variable ='ethnicity'
X_features=labeled_data_preprocessed.drop(columns=[S_variable,'h1_diasbp_max','hospital_death']).columns.to_list()

In [35]:
def run_experiment(data_name, y, S_variable, X_features,data, n_simulations):
    """
    Exécute une expérience en faisant varier un paramètre spécifique.
    
    Args:
        varying_param: Le nom du paramètre à faire varier ('t_values', 'p_mu_bias', ou 'p_sigma_bias')
        param_values: Liste des valeurs pour le paramètre variable
        fixed_params: Dictionnaire des paramètres fixes
    
    Returns:
        all_results: Dictionnaire contenant les résultats pour chaque valeur du paramètre
    """
    # Liste des modèles et métriques
    models = ['y_pred_fair', 'y_input_reg', 'y_score_equipy', 'y_pred_riken', 'y_pred_bias']
    metrics = ['r2', 'GRW2','mae', 'rmse', 'unfairness_W2', 'unfairness_W1', 'ks_stat','beta_0_NoStd','beta_NoStd','gamma_NoStd',
           'beta_0_1_Std','beta_0_2_Std','beta_1_Std','beta_2_Std','gamma_Std',
            'fair_intercept_1_NoStd', 'fair_intercept_2_NoStd', 'beta_1_NoStd','beta_2_NoStd', 
            'fair_intercept_Std','beta_Std',
            'riken_intercept_Std']
    param_riken_null=0

    # Dictionnaire pour stocker tous les résultats
    all_results = {}

    # Boucle sur chaque valeur du paramètre variable
    t_results = {model: {metric: [] for metric in metrics} for model in models}
        
    # Effectuer n_simulations pour cette valeur du paramètre
    for bootstrap in tqdm(range(n_simulations)):

            # Préparation des données
        train_dataset, test_dataset = train_test_split(data, test_size=0.2, random_state=bootstrap)
        train_dataset,pool_dataset = train_test_split(train_dataset, test_size=0.2, random_state=bootstrap)
        
        for s in data[S_variable].unique():
            # print(len(train_dataset[train_dataset[S_variable]==s]//3), 12*len(X_features))
            if len(train_dataset[train_dataset[S_variable]==s]//3) < 12*len(X_features):
                print(f'normalized_beta_{s}__and__norm_beta_{s}__are_set_to_0')
                param_riken_null +=1
            # print(len(train_dataset[train_dataset[S_variable]==s]//2),18*len(X_features))
            if len(train_dataset[train_dataset[S_variable]==s]//2) < 18*len(X_features):
                print(f'beta_{s}_bis__is_set_to_0')
                param_riken_null +=1

        y_sensitive_feature = pd.DataFrame({f"{S_variable}": test_dataset[S_variable].to_list()})
        unique_groups = test_dataset[S_variable].unique()
            
        # Modèle Fair Linear
        coef_input_model, param_dictionnary, input_model, test_dataset = Fair_model.predict_fair_linear_score(
                train_dataset, pool_dataset, test_dataset, S_variable, y, X_features, True, False, False
            )

        # Modèle EquiPy
        Benchmark_model.benchmark_equipy(train_dataset, test_dataset, 'y_input_reg', S_variable)
            
        # Modèle Riken
        dictionnary_riken_raw = Benchmark_model.riken_prediction(train_dataset, test_dataset, S_variable, X_features, y)
            
        # Modèle Evgeni (model_bias)
        Benchmark_model.weighted_group_intercepts(train_dataset, test_dataset, X_features, y, S_variable, True)
            
            # Calculer et stocker les métriques pour chaque modèle
        for prediction in models:
            t_results[prediction]['r2'].append(r2_score(test_dataset[y], test_dataset[prediction]))
            t_results[prediction]['GRW2'].append(Metrics.group_weighted_r2(test_dataset,y,prediction, S_variable))
            t_results[prediction]['rmse'].append(np.sqrt(mean_squared_error(test_dataset[y], test_dataset[prediction])))
            t_results[prediction]['unfairness_W1'].append(unfairness(np.array(test_dataset[prediction].tolist()), y_sensitive_feature))
            t_results[prediction]['unfairness_W2'].append(Metrics.unfairness_computation(prediction, S_variable, test_dataset))
            
            t_results['y_input_reg']['beta_0_NoStd'].append(param_dictionnary['beta_0'])
            t_results['y_input_reg']['beta_NoStd'].append(param_dictionnary['beta'])
            t_results['y_input_reg']['gamma_NoStd'].append(param_dictionnary['gamma'])
            t_results['y_input_reg']['beta_0_1_Std'].append(param_dictionnary['beta_0']+np.dot(param_dictionnary['empirical_mean_1'],param_dictionnary['beta']))
            t_results['y_input_reg']['beta_0_2_Std'].append(param_dictionnary['beta_0']+np.dot(param_dictionnary['empirical_mean_2'],param_dictionnary['beta']))
            t_results['y_input_reg']['beta_1_Std'].append(param_dictionnary['beta']*param_dictionnary['var_cov_product_1'])
            t_results['y_input_reg']['beta_2_Std'].append(param_dictionnary['beta']*param_dictionnary['var_cov_product_2'])
            t_results['y_input_reg']['gamma_Std'].append(param_dictionnary['gamma'])

            t_results['y_pred_fair']['fair_intercept_1_NoStd'].append(param_dictionnary['fair_intercept']-np.dot(param_dictionnary['empirical_mean_1'],param_dictionnary['beta'])/param_dictionnary['var_cov_product_1'])
            t_results['y_pred_fair']['fair_intercept_2_NoStd'].append(param_dictionnary['fair_intercept']-np.dot(param_dictionnary['empirical_mean_2'],param_dictionnary['beta'])/param_dictionnary['var_cov_product_2'])
            t_results['y_pred_fair']['beta_1_NoStd'].append(param_dictionnary['invariant_var_cov_term']*param_dictionnary['beta']/param_dictionnary['var_cov_product_1'])
            t_results['y_pred_fair']['beta_2_NoStd'].append(param_dictionnary['invariant_var_cov_term']*param_dictionnary['beta']/param_dictionnary['var_cov_product_2'])
            t_results['y_pred_fair']['gamma_NoStd'].append(0)
            t_results['y_pred_fair']['fair_intercept_Std'].append(param_dictionnary['fair_intercept'])
            t_results['y_pred_fair']['beta_Std'].append(param_dictionnary['invariant_var_cov_term'])
            t_results['y_pred_fair']['gamma_Std'].append(0)

            if len(unique_groups) >= 2:
                    ks_stat = kstest(
                        rvs=test_dataset[test_dataset[S_variable] == unique_groups[0]][prediction],
                        cdf=test_dataset[test_dataset[S_variable] == unique_groups[1]][prediction],
                        alternative='two-sided'
                    ).statistic
                    t_results[prediction]['ks_stat'].append(ks_stat)
        
        # Calculer les moyennes pour cette valeur du paramètre
    summary_t = {}
    for model in models:
        summary_t[model] = {}
        for metric in metrics:
            if t_results[model][metric]:  # Vérifier si la liste n'est pas vide
                summary_t[model][f'{metric}_mean'] = round(np.mean(t_results[model][metric]), 5)
                summary_t[model][f'{metric}_std'] = round(np.std(t_results[model][metric]), 5)
            else:
                summary_t[model][f'{metric}_mean'] = None
                summary_t[model][f'{metric}_std'] = None
        
   
        
        # Enregistrer dans MLflow pour cette valeur du paramètre
    with mlflow.start_run(run_name=f"{data_name}_{y}__{n_simulations}_simulations___{len(X_features)}_features"):
            mlflow.log_metric('param_riken_not_estimated',param_riken_null)
            # Enregistrer les moyennes et écarts-types des métriques pour chaque modèle
            for model in models:
                for metric in metrics:
                    mean_key = f'{metric}_mean'
                    std_key = f'{metric}_std'
                    
                    if mean_key in summary_t[model] and summary_t[model][mean_key] is not None:
                        mlflow.log_metric(f"{model}_{mean_key}", summary_t[model][mean_key])
                    
                    if std_key in summary_t[model] and summary_t[model][std_key] is not None:
                        mlflow.log_metric(f"{model}_{std_key}", summary_t[model][std_key])
    
    return summary_t, t_results


In [38]:
summary_t,t_results= run_experiment('GOSSIS_GRW2',y, S_variable, X_features, labeled_data_preprocessed, 50)

100%|██████████| 50/50 [20:56<00:00, 25.13s/it]
